# PRACTICA DE ML CON REGRESION#

Para esta practica de aprendizaje automatico con regresión, vamos a usar el aumento de gradiente extremo, o XGB  y a compararlo con la maquina de aumento de gradiente ligero, o LGBM, por sus siglas en ingles

Los pasos a seguir son:
1. Cargar las librer'ias y los datos con pandas
2. Hacer una exploración de los datos 
3. Separar los datos de entrenamiento en entrenamiento y validación
4. Entrenar los modelos con los datos de entrenamiento y usar los datos de validación para evaluar nuestro entrenamiento
5. Seleccionar el mejor modelo
6. Subir los resultados a la competición

# 1. Cargamos las librerías y los datos# 

El primer paso es cargar las librerias necesarias para este trabajo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb

Cargamos los datos de entrenamiento y mostramos las cinco primeras filas

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
train_df.head()

Mostramos los datos de test y mostramos las cinco primeras filas

In [ ]:
test_df = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
test_df.head()

# 2. Hacemos una exploración de los datos

Primero comprobamos que no haya datos faltantes

In [ ]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

No faltan datos en las columnas

Vemos como están correlacionados los datos entre sí

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(train_df.corr(), annot = True)

Las columnas que mas condicionan el valor final son: cont2, cont3, cont7 y cont11

Ponemos en una gráfica como se distribuye la variable 'target'

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))
sns.distplot(train_df['target'])
ax.xaxis.grid(True)
ax.set(ylabel = "Valores")
ax.set(xlabel = "Target")
ax.set(title = "Distribuicion de target")
sns.despine(trim = True, left = True)
plt.show()

Vemos que la distribuición de datos de la variable se mueve entre 6 y 10

Con el siguiente codigo vamos a var como se ditribuyen los valores de train.

primero tomamos las columans de train_df, nuestro conjunto de entrenamiento pero sin la columna objetivo target ni la de identificacion id, despues, creamos una hoja que pueda ser usada para dibujar las 14 columnas de los dos conjuntos. Tras esto, dibujamos las graficas y las mostramos por pantalla

In [ ]:
cols = [col for col in train_df.columns if col not in ['id','target']]

fig, ax = plt.subplots(nrows = 7, ncols = 2, figsize = (15,15))
fig.tight_layout()

i = j = 0
for col in cols:
    sns.kdeplot(data = train_df[col],shade=True,ax=ax[i][j],legend=False)
    ax[i][j].set_title(col)
    j += 1
    if(j%2 == 0):
        i += 1
        j = 0
        
plt.show()

# 3. Separamos los datos de entrenamiento y validación

Esto lo hacemos eliminado la variable 'target' del conjunto de entrenamiento y dándosela a una variable y, el resto lo metemos en una variable X, también eliminamos la columna 'id'. Creamos una variable X_test eliminando la columna 'id' del conjunto test. Tras esto, mostramos el tamaño de los conjuntos train y val

In [ ]:
y = train_df['target']
X = train_df.drop(['id', 'target'], axis = 1)
X_test = test_df.drop(['id'], axis = 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 597)

In [ ]:
print(X_train.shape)
print(X_val.shape)

# 4. Entrenamos los modelos con los datos de entrenamiento y usamos los datos de validación para evaluar nuestro entrenamiento

El primer sistema que vamos a usar es el XGB, para ello, primero definimos los parametros que vamos a usar

In [ ]:
params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "max_depth": 5,
    "eta": 0.05,
    "random_state": 751
}

In [ ]:
d_train = xgb.DMatrix(X_train, label = y_train)
d_val = xgb.DMatrix(X_val, label = y_val)
d_test = xgb.DMatrix(X_test)

In [ ]:
model_1 = xgb.train(params = params,
                    dtrain = d_train,
                    num_boost_round = 10000,
                    early_stopping_rounds = 20,
                    verbose_eval = 10,
                    evals = [(d_train, "train"), (d_val, "val")])

In [ ]:
predict_1 = model_1.predict(d_val, ntree_limit = model_1.best_ntree_limit)
rmse_model_1 = np.sqrt(mean_squared_error(y_val, predict_1))
rmse_model_1

In [ ]:
predict_1 = model_1.predict(d_test, ntree_limit = model_1.best_ntree_limit)
predict_1

In [ ]:
model_2 = lgb.LGBMRegressor(random_state = 100, 
                        n_estimators = 500, 
                        min_data_per_group = 5, 
                        boosting_type = 'gbdt',
                        num_leaves = 128, 
                        learning_rate = 0.005, 
                        subsample_for_bin = 200000, 
                        importance_type ='split', 
                        metric ='rmse', 
                        min_data_in_leaf = 50)

model_2.fit(X_train, y_train)

In [ ]:
predict_2 = model_2.predict(X_val)
predict_2

In [ ]:
rmse_model_2 = np.sqrt(mean_squared_error(y_val, predict_2))
rmse_model_2

# 5. Seleccionar el mejor modelo

Como podemos ver por el valor que nos devuelve el RMSE (Mean squared error regression loss), el mejopr modelo es el de lgb, con un 70% e acierto frente al 69% que nos da el xgb

# 6. Subir los resultados a la competición

In [ ]:
test_predict = model_2.predict(X_test)
test_predict

In [ ]:
output = pd.DataFrame({"id":X_test.index,'target':test_predict})
output

In [ ]:
output.to_csv('./Submission.csv',index=False)